In [2]:
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

04/10/2021 17:34:18 - INFO - faiss -   Loading faiss.
04/10/2021 17:34:20 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [17]:
from haystack.document_store.faiss import FAISSDocumentStore

document_store = FAISSDocumentStore(faiss_index_factory_str="HNSW")

04/10/2021 18:05:22 - INFO - haystack.document_store.faiss -   HNSW params: n_links: 128, efSearch: 20, efConstruction: 80


In [12]:
from lxml import etree
from lxml.etree import tostring

input_file_path = '/Users/sava/Code/FitBot/web-app/static/Wikipedia-Strength-Training.xml'

tree = etree.parse(input_file_path)
root = tree.getroot()

document_dictionaries = []
# iterate through all the titles
for text_node in root.findall(".//text", namespaces=root.nsmap)[:20]:
    document_dictionaries.append({'text': text_node.text[:200], 'meta': None})

document_store.write_documents(document_dictionaries)

In [18]:
from haystack.retriever.dense import DensePassageRetriever
retriever = DensePassageRetriever(document_store=document_store,
                                  query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
                                  passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
                                  max_seq_len_query=64,
                                  max_seq_len_passage=256,
                                  batch_size=16,
                                  use_gpu=True,
                                  embed_title=True,
                                  use_fast_tokenizers=True)
# document_store.update_embeddings(retriever)

In [14]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)


04/10/2021 17:43:55 - INFO - farm.utils -   Using device: CPU 
04/10/2021 17:43:55 - INFO - farm.utils -   Number of GPUs: 0
04/10/2021 17:43:56 - INFO - farm.utils -   Distributed Training: False
04/10/2021 17:43:56 - INFO - farm.utils -   Automatic Mixed Precision: None
Some weights of RobertaModel were not initialized from the model checkpoint at deepset/roberta-base-squad2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
04/10/2021 17:44:12 - WARNING - farm.utils -   ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
04/10/2021 17:44:12 - INFO - farm.utils -   Using device: CPU 
04/10/2021 17:44:12 - INFO - farm.utils -   Number of GPUs: 0
04/10/2021 17:44:12 - INFO - farm.utils -   Distributed Training: False
04/10/2021 17:44:12 - INFO - farm.utils -   Automatic Mixed Precision: None
04/10/2021 17

In [15]:
from haystack.pipeline import ExtractiveQAPipeline
pipe = ExtractiveQAPipeline(reader, retriever)

In [19]:
prediction = pipe.run(query="What is stamina?", top_k_retriever=10, top_k_reader=5)
print_answers(prediction, details="minimal")

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.29 Batches/s]

[   {   'answer': 'Physical fitness',
        'context': '[Category:Determinants of health]]\n'
                   '[[Category:Hobbies]]\n'
                   '[[Category:Management of obesity]]\n'
                   '[[Category:Physical fitness]]\n'
                   '[[Category:Recreation]]\n'
                   '[[Cat'},
    {   'answer': 'Endurance training',
        'context': '{{Other uses}}\n'
                   '[[File:Endurance training for Mt Carstensz '
                   'Pyramid.JPG|thumb|Twins doing endurance trek at the '
                   'foothills of Himalayas]]\n'
                   '{{short descrip'},
    {   'answer': 'Physical activity',
        'context': '[[File:TaipeiMetro-Cleaner.JPG|thumb|Physical activity is '
                   'not just [[exercise]]. It includes other activities that '
                   'involve movement; for example clean'},
    {   'answer': "Cardiorespiratory fitness''' ('''CRF''') refers to the "
                  'ability of the [[